### Training and Testing 2

In [22]:
import numpy as np
import os
import constants as cs
import yaml
import globali as gg
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [23]:
videos_for_action = {}

# instogram classes from dataset2
actionlist = os.listdir("./dataset2")
for action in actionlist:
    videos_for_action[action] = list()
    videos = os.listdir("./dataset2/" + action)
    count = 0
    for video in videos:
        video_path = os.path.join("dataset2", action, video)
        videos_for_action[action].append(video_path)

videos_for_action = dict(sorted(videos_for_action.items(), key=lambda item: len(item[1]), reverse=True))
print(videos_for_action)


{'thin': ['dataset2\\thin\\57936', 'dataset2\\thin\\57940', 'dataset2\\thin\\57939', 'dataset2\\thin\\66606', 'dataset2\\thin\\57937', 'dataset2\\thin\\57949', 'dataset2\\thin\\57919', 'dataset2\\thin\\57943', 'dataset2\\thin\\57933', 'dataset2\\thin\\57934', 'dataset2\\thin\\57948', 'dataset2\\thin\\57935', 'dataset2\\thin\\57947', 'dataset2\\thin\\66607', 'dataset2\\thin\\57941', 'dataset2\\thin\\57942'], 'go': ['dataset2\\go\\24857', 'dataset2\\go\\24956', 'dataset2\\go\\65824', 'dataset2\\go\\24940', 'dataset2\\go\\24946', 'dataset2\\go\\24943', 'dataset2\\go\\24960', 'dataset2\\go\\24961', 'dataset2\\go\\24962', 'dataset2\\go\\24947', 'dataset2\\go\\24941', 'dataset2\\go\\24955', 'dataset2\\go\\24952', 'dataset2\\go\\69345', 'dataset2\\go\\24954'], 'help': ['dataset2\\help\\27208', 'dataset2\\help\\27217', 'dataset2\\help\\27216', 'dataset2\\help\\27209', 'dataset2\\help\\65891', 'dataset2\\help\\65889', 'dataset2\\help\\27214', 'dataset2\\help\\27206', 'dataset2\\help\\27215', 'd

In [24]:
actions_and_videos_used = dict(list(videos_for_action.items())[:10])

actions = []

for key, value in actions_and_videos_used.items():
    actions.append(key)
    
display(actions_and_videos_used)

{'thin': ['dataset2\\thin\\57936',
  'dataset2\\thin\\57940',
  'dataset2\\thin\\57939',
  'dataset2\\thin\\66606',
  'dataset2\\thin\\57937',
  'dataset2\\thin\\57949',
  'dataset2\\thin\\57919',
  'dataset2\\thin\\57943',
  'dataset2\\thin\\57933',
  'dataset2\\thin\\57934',
  'dataset2\\thin\\57948',
  'dataset2\\thin\\57935',
  'dataset2\\thin\\57947',
  'dataset2\\thin\\66607',
  'dataset2\\thin\\57941',
  'dataset2\\thin\\57942'],
 'go': ['dataset2\\go\\24857',
  'dataset2\\go\\24956',
  'dataset2\\go\\65824',
  'dataset2\\go\\24940',
  'dataset2\\go\\24946',
  'dataset2\\go\\24943',
  'dataset2\\go\\24960',
  'dataset2\\go\\24961',
  'dataset2\\go\\24962',
  'dataset2\\go\\24947',
  'dataset2\\go\\24941',
  'dataset2\\go\\24955',
  'dataset2\\go\\24952',
  'dataset2\\go\\69345',
  'dataset2\\go\\24954'],
 'help': ['dataset2\\help\\27208',
  'dataset2\\help\\27217',
  'dataset2\\help\\27216',
  'dataset2\\help\\27209',
  'dataset2\\help\\65891',
  'dataset2\\help\\65889',
  'data

In [25]:
print(actions)

['thin', 'go', 'help', 'before', 'bowling', 'computer', 'bed', 'cool', 'accident', 'tall']


In [26]:
sequences, labels = [], []

#for action in tqdm(list(actions_and_videos_used), desc="Processing Actions"):
for action in list(actions_and_videos_used):
    for video in actions_and_videos_used[action]:
        window = []
        missing = 25-len(os.listdir(video))
        npy_path_last = None
        for npy_file in os.listdir(video):
            npy_path = os.path.join(video, npy_file)
            res = np.load(npy_path)
            window.append(res)
            npy_path_last = npy_path
        for i in range(missing):
            res = np.load(npy_path_last)
            window.append(res)
        #print(len(window))
        sequences.append(window)
        labels.append(actions.index(action))
        

In [27]:
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]


In [28]:
X = np.array(sequences)
X.shape

(139, 25, 1692)

In [29]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
print(y_train.shape)
print(y_test.shape)

(132, 10)
(7, 10)


### Training

In [31]:
def create_folder_for_training(number):
    # create folders
    log_f = os.path.join(cs.LOGS_FOLDER, 'log_' + str(number), '')
    # log_f = os.path.join(os.getcwd(), 'logs')
    #log_f = ""
    # log_f = cs.LOGS_FOLDER + "" + "log_" + str(number)
    model_f = os.path.join(cs.MODELS_FOLDER, 'model_' + str(number))
    # train_f = os.path.join(cs.TRAINING_FOLDER, 'training_' + str(number))
    os.makedirs(log_f, exist_ok=True)
    os.makedirs(model_f, exist_ok=True)
    log_f = "D:\logs" # + "log_" + str(number)
    return log_f, model_f

In [32]:
log_f, model_f = create_folder_for_training(4)

In [33]:
# log
tensorboard_callback = TensorBoard(log_dir=log_f, histogram_freq=1)
training_log = os.path.join(log_f, 'train_log.csv')
csv_logger = CSVLogger(training_log, append=True)

In [34]:
# LSTM (model_4)

#np_actions = np.array(actions)
#print(np_actions.shape[0])

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(25, 1692)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(len(actions), activation='softmax'))

In [35]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
early_stopping = EarlyStopping(monitor='loss', patience=60, restore_best_weights=True)
best_model_path = os.path.join(model_f, 'best_model.h5') 
checkpoint = ModelCheckpoint(best_model_path, monitor='loss', save_best_only=True)
history = model.fit(X_train, y_train, epochs=3000, callbacks=[early_stopping,
                                                              checkpoint,
                                                              csv_logger,
                                                              tensorboard_callback])
# save best model
final_model_path = os.path.join(model_f, 'final_model.h5')
model.save(final_model_path)


Epoch 1/3000
5/5 [==============================] - ETA: 0s - loss: 3.3746 - categorical_accuracy: 0.0985

C:\Users\Paolo\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - 21s 256ms/step - loss: 3.3746 - categorical_accuracy: 0.0985
Epoch 2/3000
5/5 [==============================] - 1s 267ms/step - loss: 3.3104 - categorical_accuracy: 0.1136
Epoch 3/3000
5/5 [==============================] - 1s 279ms/step - loss: 3.2615 - categorical_accuracy: 0.1061
Epoch 4/3000
5/5 [==============================] - 1s 259ms/step - loss: 3.2045 - categorical_accuracy: 0.1061
Epoch 5/3000
5/5 [==============================] - 1s 267ms/step - loss: 3.1573 - categorical_accuracy: 0.1212
Epoch 6/3000
5/5 [==============================] - 1s 254ms/step - loss: 3.1141 - categorical_accuracy: 0.1136
Epoch 7/3000
5/5 [==============================] - 1s 297ms/step - loss: 3.0739 - categorical_accuracy: 0.1061
Epoch 8/3000
5/5 [==============================] - 1s 281ms/step - loss: 3.0287 - categorical_accuracy: 0.0985
Epoch 9/3000
5/5 [==============================] - 1s 269ms/step - loss: 2.9974 - categorical_accuracy: 0.0909
Epoc

KeyboardInterrupt: 